In [6]:
import os
from os.path import join as oj
import sys, time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data_pecarn
import data_psrc
import data
from data import classification_setup, get_feat_names

import matplotlib.gridspec as grd

# sns.set(style="black")
# plt.style.use('dark_background')
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# look at all dsets

In [7]:
df = data_psrc.get_data()
feat_names, psrc_feats = get_feat_names(df)
ks = [k for k in feat_names if not 'time' in k.lower() and not 'other' in k.lower()]

In [9]:
d = df[ks]

In [ ]:
d

### eda plots

In [65]:
fnames = df.keys()
missing_arr = np.zeros((len(fnames), NUM_PATIENTS + 1), dtype=np.bool)

for i, fname in enumerate(fnames):
#     df = r[fname]
    ids = np.unique(df.id.values)
#     print(np.min(ids), np.max(ids))
    missing_arr[i, ids] = 1

In [ ]:
plt.figure(figsize=(5, 10))
sns.barplot(x=missing_arr.sum(axis=1), y=fnames, orient='h')
plt.xlabel('number of patients')
plt.show()

In [ ]:
# plt.imshow(not_missing)
plt.figure(figsize=(12, 20), dpi=300)
sns.clustermap(missing_arr, yticklabels=fnames)
# plt.tight_layout()
plt.show()

In [ ]:
# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(10, 220, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
# sns.clustermap(corr) #, mask=mask, cmap=cmap, vmax=.3, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})

# look at demographics

In [46]:
# demographics = pd.read_csv('iaip_data/Datasets/demographics.csv').rename(columns={'SubjectID': 'id'})
# ids_np = np.array(list(ids)) - 1
ids_np = df.iai_intervention == 1
# vals = demographics.values[ids_np]
# print(list(demographics))
print(df.keys()[:10])

Index(['Race', 'Hispanic ethnicity',
       'Mechanism of injury (choice=Assault/struck)',
       'Mechanism of injury (choice=ATV injury)',
       'Mechanism of injury (choice=Bike crash)',
       'Mechanism of injury (choice=Bike struck by auto)',
       'Mechanism of injury (choice=Fall > 10 ft. height)',
       'Mechanism of injury (choice=Golf cart injury)',
       'Mechanism of injury (choice=Motorcycle/dirt bike crash)',
       'Mechanism of injury (choice=MVC)'],
      dtype='object')


In [53]:
# the mean age is the only one which is wrong
print('age', np.mean(df.ageinyrs[ids_np]), np.mean(df.ageinyrs))
print('age<2', np.sum(df.ageinyrs[ids_np] < 2), np.sum(df.ageinyrs < 2))
# print('sex', np.sum(df.SEX[ids_np] == 'M'), np.sum(df.SEX == 'M'))
# print('hispanic ethnicity', np.sum(df.HISPANIC_ETHNICITY[ids_np] == 0), np.sum(df.HISPANIC_ETHNICITY == 0))

age 7.244623655913979 7.7972653869591735
age<2 6 216
